# Install

In [1]:
!pip3 install -q opyplus

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# Simulation

In [2]:
import os 
import opyplus as op
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### Select version of EnergyPlus

In [3]:
eplus_dir_path = op.get_eplus_base_dir_path((9, 3, 0))

### Prepare input paths and run simulation.

In [4]:
# idf path (building model)
idf_path = os.path.join(
    eplus_dir_path,
    "ExampleFiles",
    "1ZoneEvapCooler.idf"
)

# epw path (weather)
epw_path = os.path.join(
    eplus_dir_path,
    "WeatherData",
    "USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
)

# run simulation
s = op.simulate(idf_path, epw_path, "my-first-simulation")

called Simulation.from_input on a simulation directory that is not empty (my-first-simulation)


### Check status, and inspect EPlus .err file.

In [5]:
print(f"status: {s.get_status()}\n")
print(f"Eplus .err file:\n{s.get_out_err().get_content()}")

status: finished

Eplus .err file:
Program Version,EnergyPlus, Version 9.3.0-baff08990c, YMD=2021.06.01 15:11,
   ** Warning ** Weather file location will be used rather than entered (IDF) Location object.
   **   ~~~   ** ..Location object=DENVER CENTENNIAL CO USA WMO=724666
   **   ~~~   ** ..Weather File Location=San Francisco Intl Ap CA USA TMY3 WMO#=724940
   **   ~~~   ** ..due to location differences, Latitude difference=[2.12] degrees, Longitude difference=[17.22] degrees.
   **   ~~~   ** ..Time Zone difference=[1.0] hour(s), Elevation difference=[99.89] percent, [1791.00] meters.
   ** Warning ** SetUpDesignDay: Entered DesignDay Barometric Pressure=81560 differs by more than 10% from Standard Barometric Pressure=101301.
   **   ~~~   ** ...occurs in DesignDay=DENVER CENTENNIAL ANN HTG 99.6% CONDNS DB, Standard Pressure (based on elevation) will be used.
   ** Warning ** GetAirPathData: AirLoopHVAC="EVAP COOLER SYSTEM" has no Controllers.
   ** Warning ** SetUpDesignDay: Ente

### Retrieve and display outputs.

In [6]:
# retrieve hourly output (.eso file)
hourly_output = s.get_out_eso()

# ask for datetime index on year 2013
hourly_output.create_datetime_index(2013)

# get Pandas dataframe
df = hourly_output.get_data()

# monthly resample and display
# print(df[[
#     "environment,Site Outdoor Air Drybulb Temperature",
#     "main zone,Zone Mean Air Temperature"
# ]].resample("MS").mean())
df.head()

,month,day,hour,minute,end_minute,dst,day_type,"environment,Site Outdoor Air Drybulb Temperature","environment,Site Outdoor Air Wetbulb Temperature","environment,Site Outdoor Air Humidity Ratio",...,"supply outlet node,System Node Temperature","supply outlet node,System Node Mass Flow Rate","outside air inlet node,System Node Temperature","main zone outlet node,System Node Temperature","main zone node,System Node Temperature","main zone inlet node,System Node Temperature","zone equipment inlet node,System Node Temperature","zone equipment outlet node,System Node Temperature","main zone inlet node atinlet,System Node Temperature","relief air outlet node,System Node Temperature"
2013-01-01 00:00:00,1,1,0,0,60,0,Tuesday,6.991667,5.985290,0.005329,...,20.0,0.0,6.991667,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2013-01-01 01:00:00,1,1,1,0,60,0,Tuesday,7.200000,6.441589,0.005617,...,20.0,0.0,7.200000,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2013-01-01 02:00:00,1,1,2,0,60,0,Tuesday,6.908333,6.114196,0.005469,...,20.0,0.0,6.908333,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2013-01-01 03:00:00,1,1,3,0,60,0,Tuesday,6.350000,5.714319,0.005371,...,20.0,0.0,6.350000,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2013-01-01 04:00:00,1,1,4,0,60,0,Tuesday,5.108333,4.782518,0.005143,...,20.0,0.0,5.108333,20.0,20.0,20.0,20.0,20.0,20.0,20.0


## EPlus Model (idf file)

### Load Energy Plus model.

In [7]:
# idf path
idf_path = os.path.join(
    eplus_dir_path,
    "ExampleFiles",
    "1ZoneEvapCooler.idf"
)

# load epm object
epm = op.Epm.from_idf(idf_path)

### Iter constructions:

In [8]:
for construction in epm.Construction:
    print(construction)

Construction,
    r13wall,                       ! Name
    r13layer;                      ! Outside Layer
Construction,
    floor,                         ! Name
    c5 - 4 in hw concrete;         ! Outside Layer
Construction,
    roof31,                        ! Name
    r31layer;                      ! Outside Layer


### Retrieve concrete material.

In [9]:
concrete = epm.Material.one("c5 - 4 in hw concrete")
print(concrete)

Material,
    c5 - 4 in hw concrete,         ! Name
    mediumrough,                   ! Roughness
    0.1014984,                     ! Thickness
    1.729577,                      ! Conductivity
    2242.585,                      ! Density
    836.8,                         ! Specific Heat
    0.9,                           ! Thermal Absorptance
    0.65,                          ! Solar Absorptance
    0.65;                          ! Visible Absorptance


### Change thickness and conductivity.

In [10]:
# change thickness and conductivity
concrete.thickness = 0.2
concrete.conductivity = 1.5

# print new values
print(concrete)

# save new idf
epm.save("my-first-model.idf")

Material,
    c5 - 4 in hw concrete,         ! Name
    mediumrough,                   ! Roughness
    0.2,                           ! Thickness
    1.5,                           ! Conductivity
    2242.585,                      ! Density
    836.8,                         ! Specific Heat
    0.9,                           ! Thermal Absorptance
    0.65,                          ! Solar Absorptance
    0.65;                          ! Visible Absorptance


### Outputs (eso file)

Connect to previous simulation and retrieve eso object.

In [11]:
s = op.Simulation("my-first-simulation")
eso = s.get_out_eso()

### Display what is contained in eso file (environments, variable names and frequencies).

In [12]:
print(eso.get_info())

Standard output
  environments
    denver centennial ann htg 99.6% condns db (0)
      latitude: 37.62
      longitude: -122.4
      timezone_offset: -8.0
      elevation: 2.0
    denver centennial ann clg 1% condns db=>mwb (1)
      latitude: 37.62
      longitude: -122.4
      timezone_offset: -8.0
      elevation: 2.0
    run period 1 (2)
      latitude: 37.62
      longitude: -122.4
      timezone_offset: -8.0
      elevation: 2.0
  variables
    hourly
      environment,Site Outdoor Air Drybulb Temperature (7)
      environment,Site Outdoor Air Wetbulb Temperature (8)
      environment,Site Outdoor Air Humidity Ratio (9)
      environment,Site Outdoor Air Relative Humidity (10)
      main zone,Zone Mean Air Temperature (11)
      main zone baseboard,Baseboard Electric Power (196)
      supply inlet node,System Node Temperature (492)
      fan inlet node,System Node Temperature (493)
      evap cooler inlet node,System Node Temperature (494)
      supply outlet node,System Node Tem

Natively, outputs don’t have a year and their indexes are not stored in datetimes (but in tuples of integers instead: month, day, hour). We transform outputs to datetime index dataframes to ease future analysis (datetimes are easy to manipulate, for example for resample operations).

In [13]:
eso.create_datetime_index(2013)  # we indicate the year

### Explore window design day data : display mean daily exterior and interior temperatures.

winter_design_day_df = eso.get_data("denver centennial ann htg 99.6% condns db")
winter_design_day_df[[
    "main zone,Zone Mean Air Temperature",
    "environment,Site Outdoor Air Drybulb Temperature"
    ]].resample("D").mean()


### Explore run period data : display mean daily exterior and interior temperatures.

In [14]:
# default environment is the last one found, which is the run period environment in our case
run_period_df = eso.get_data()

# daily resample
daily_df = run_period_df[[
    "main zone,Zone Mean Air Temperature",
    "environment,Site Outdoor Air Drybulb Temperature"
    ]].resample("D").mean()

# display
daily_df.head()  # will only display first rows of dataframe

,"main zone,Zone Mean Air Temperature","environment,Site Outdoor Air Drybulb Temperature"
2013-01-01,20.058164,8.704167
2013-01-02,20.035524,9.857639
2013-01-03,20.085615,12.200000
2013-01-04,19.999999,8.456250
2013-01-05,20.000000,7.819097


# Weather data (epw file)

### Load Weather data.

In [15]:
# epw path
epw_path = os.path.join(
    eplus_dir_path,
    "WeatherData",
    "USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"
)

# load weather data object
weather_data = op.WeatherData.from_epw(epw_path)

In [16]:
print(weather_data.get_info())

WeatherData
	has datetime instants: False
	latitude: 37.62
	longitude: -122.40
	timezone_offset: -8.0
	elevation: 2.0
	data period: 1999-01-01T00:00:00, 1997-12-31T23:00:00


In [17]:
weather_data.create_datetime_instants(2013)  # we indicate start year

# check that operation worked
print(f"has datetime index: {weather_data.has_datetime_instants}")

has datetime index: True


### Retrieve weather series data (pandas dataframe object).

In [18]:
df = weather_data.get_weather_series()

# print columns
print(f"columns: {list(sorted(df.columns))}\n")

# print drybulb first rows
print("drybulb:")
print(df["drybulb"].head())

columns: ['Albedo', 'aerosol_opt_depth', 'atmos_pressure', 'ceiling_hgt', 'datasource', 'day', 'days_last_snow', 'dewpoint', 'difhorillum', 'difhorrad', 'dirnorillum', 'dirnorrad', 'drybulb', 'extdirrad', 'exthorrad', 'glohorillum', 'glohorrad', 'horirsky', 'hour', 'liq_precip_depth', 'liq_precip_rate', 'minute', 'month', 'opaqskycvr', 'precip_wtr', 'presweathcodes', 'presweathobs', 'relhum', 'snowdepth', 'totskycvr', 'visibility', 'winddir', 'windspd', 'year', 'zenlum']

drybulb:
2013-01-01 01:00:00    7.2
2013-01-01 02:00:00    7.2
2013-01-01 03:00:00    6.7
2013-01-01 04:00:00    6.1
2013-01-01 05:00:00    4.4
Freq: H, Name: drybulb, dtype: float64


### Add one degree celcius to drybulb and set new weather series.

In [19]:
# add one degree
df["drybulb"] += 1  # equivalent of df["drybulb"] = df["drybulb"] + 1

# set new dataframe
weather_data.set_weather_series(df)

# check it worked
print(weather_data.get_weather_series()["drybulb"].head())

2013-01-01 01:00:00    8.2
2013-01-01 02:00:00    8.2
2013-01-01 03:00:00    7.7
2013-01-01 04:00:00    7.1
2013-01-01 05:00:00    5.4
Freq: H, Name: drybulb, dtype: float64


### Save data in a new epw file.

In [20]:
# save new epw
weather_data.to_epw("one-more-drybulb-degree.epw")